In [208]:
import numpy as np
import pandas as pd
import random
import time
from sklearn.model_selection import RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import linalg
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [209]:
Edges = pd.read_csv("/Users/qizhe/Documents/GitHub/GraphNN/Data/DD244/DD244 Edges.csv",header=None)
Edges = np.array(Edges)

In [210]:
Labels = pd.read_csv("/Users/qizhe/Documents/GitHub/GraphNN/Data/DD244/DD244 node_labels.csv",header=None)
y = np.array(Labels)

In [211]:
A = np.zeros((291,291))
for i in range (0,822):
    A[Edges[i,0]-1,Edges[i,1]-1]=1

In [212]:
n=291
k=int(y.max())
acc_ase=[]
k

20

In [199]:
def get_acc(model,X_train,X_test,y_train,y_test):
    model.fit(X_train,y_train.ravel())
    return model.score(X_test,y_test.ravel())

In [205]:
begin1 = time.time()
U, s, VT = np.linalg.svd(A)
end1 = time.time()
newx=([])
dmax=50
tmp = np.zeros([dmax,1])
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
for train_index, test_index in rkf.split(A):
     #print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = A[train_index], A[test_index]
     y_train, y_test = y[train_index], y[test_index]
     X1 = X_train.transpose()
     X_train, X_test = X1[train_index], X1[test_index]
     for d in range (1,dmax):
        newx=U[:,0:d]*s[0:d]**0.5
        #newx=np.reshape(newx,(n,d))
        clf2 = LinearDiscriminantAnalysis()
        newx_train, newx_test =newx[train_index], newx[test_index]
        clf2.fit(newx_train, y_train.ravel())
        tmp[d,0]=tmp[d,0]+clf2.score(newx_test, y_test.ravel())/50
db=tmp.argmax()
ASELDAACC=tmp.max()
ASELDATIME=end1-begin1
print(ASELDAACC)
print(ASELDATIME)

0.0955230859146698
0.04063677787780762


In [200]:
global k
nk = np.zeros((1,k))
AEEKNNACC=[]
AEELDAACC=[]
for i in range(0,len(y_train)-1):
    nk[0,int(y_train[i,0]-1)]=nk[0,int(y_train[i,0]-1)]+1
w = np.zeros((int(np.size(y_train)),k))
for i in range(0,int(np.size(y_train))):
    k=int(y_train[i])
    w[i][k-1]=1/nk[0,k-1]
for train_index, test_index in rkf.split(A):
    #print("TRAIN:", train_index, "TEST:", test_index)
     z_train=np.matmul(X_train,w)
     z_test = np.matmul(X_test,w)
     neigh = KNeighborsClassifier(weights='uniform',metric='euclidean')
     neigh.fit(z_train, y_train.ravel())
     AEEKNNACC.append(neigh.score(z_test, y_test))
     clf4 = LinearDiscriminantAnalysis()
     clf4.fit(z_train,y_train.ravel())
     AEELDAACC.append(clf4.score(z_test, y_test.ravel()))

In [213]:
%run Encoder.ipynb
EncoderLDA(A)

d = 16
Accuracy for each method: [0.09552309 0.10344828 0.05172414]
Time for each method: [0.03059697 0.00513887 0.00314212]
